In this notebook we will test the candidate generation module 

In [1]:
# IMPORTs
from utils.task import Task
import json
import os
from src.pipeline.candidate_generation import candidate_generation
from dotenv import load_dotenv
from utils.prompt import load_prompt
import tiktoken
from tqdm import tqdm

load_dotenv()

True

In [2]:
# Function to load JSON data
def load_json_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data


# Function to create task object
def create_task(example):
    return Task(example)

In [3]:
# load the task data
filepath = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/subsampled_test.json"
data = load_json_data(filepath)
# load the retrieved entities
filepath_entities = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_entities.json"
retrieved_entities = load_json_data(filepath_entities)
# load the retrieved context
filepath_context = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_context.json"
retrieved_context = load_json_data(filepath_context)
# load the selected schema
filepath_schema = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/selected_schema.json"
selected_schema = load_json_data(filepath_schema)

In [4]:
# test candidate generation module with a random sample
index = 7
example = data[index]
task = create_task(example)

model = ("gpt-4")
ans = candidate_generation(task=task, retrieved_entities=retrieved_entities[index],
                           retrieved_context=retrieved_context[index], selected_schema=selected_schema[index],
                           model=model,
                           num_samples=1)
ans

{'chain_of_thought_reasoning': "To answer the question, I need to find the full name and college of a club member who holds the position of 'Secretary'. First, I will identify the relevant tables and columns. The 'club_member' table likely contains columns for member names and positions, and another table, possibly named 'college', should link members to their respective colleges. I will use a JOIN operation to combine these tables based on a common key, typically a member ID. The WHERE clause will filter the position to 'Secretary'. Since the question specifies to list the full name and college, I will select the 'first_name', 'last_name', and the college name from the joined tables. I will ensure to follow the database admin instructions, particularly focusing on not concatenating the name columns in the SELECT clause and using aliases for tables.",
 'SQL': "SELECT T1.first_name, T1.last_name, T2.college_name FROM club_member AS T1 JOIN college AS T2 ON T1.college_id = T2.id WHERE T1

In [5]:
#test candidate generation module with the subsampled test
model = "gpt-4"
num_samples = 1
res = []
for index, sample in tqdm(enumerate(data)):
    task = create_task(sample)
    ans = candidate_generation(task=task, retrieved_entities=retrieved_entities[index],
                               retrieved_context=retrieved_context[index], selected_schema=selected_schema[index],
                               model=model,
                               num_samples=num_samples)
    res.append(ans)
    print(str(index) + " : ", ans)

1it [00:08,  8.39s/it]

0 :  {'chain_of_thought_reasoning': "To answer the question, we need to identify the atom IDs associated with the molecule 'TR346' and count the distinct types of bonds that can be created by this molecule. First, we need to find the table that links molecules to their atom IDs, which is likely a molecule_atoms table. We then join this table with a bonds table that should contain information about the types of bonds each atom can form. We filter the molecule_atoms table for the molecule 'TR346', join it with the bonds table on atom_id, and then count the distinct bond types. We use GROUP BY to group results by molecule_id, ensuring we count bond types per molecule. The SQL query will select the atom_id and the count of distinct bond types, ensuring to filter out any null values in the bond type column to comply with the database admin instructions.", 'SQL': "SELECT T1.atom_id, COUNT(DISTINCT T2.bond_type) AS bond_type_count FROM molecule_atoms T1 JOIN bonds T2 ON T1.atom_id = T2.atom_i

2it [00:18,  9.45s/it]

1 :  {'chain_of_thought_reasoning': "To answer the question, I need to find a school that meets several criteria: it must be a high school (EILCode = 'HS'), located in Merced county, with Lunch Provision 2, and the grades it covers should range from 9 to 12. I will start by identifying the relevant tables and columns based on the schema and hints provided. The EILCode will help identify the school level, and I will use the columns for county, lunch provision, lowest grade, and highest grade to filter the schools accordingly. Once the correct school is identified, I will select the city location of this school. I will use aliases for tables as required and ensure to filter out any null values in columns used for logical operations or sorting. The final step is to write the SQL query following the database admin instructions, ensuring to only select the city location column as requested.", 'SQL': "SELECT T1.city_location FROM Schools AS T1 WHERE T1.EILCode = 'HS' AND T1.county = 'Merced'

3it [00:24,  7.61s/it]

...

12 :  {'chain_of_thought_reasoning': "To solve the query, I need to calculate the percentage of bonds that are double bonds (' = ') among all bonds in chemical compounds containing the molecule TR047. First, I'll identify the relevant tables and columns: molecule_id and bond_type from a bonds table. I'll filter the bonds by molecule_id = 'TR047'. Then, I'll use a CASE statement to count only the bonds where bond_type is ' = '. The percentage is calculated by dividing the count of double bonds by the total count of bonds for TR047, and then multiplying by 100 to get the percentage. I'll ensure to handle NULL values in bond_type by filtering them out. The final query will use a SELECT statement to perform these calculations and output the required percentage.", 'SQL': "SELECT (SUM(CASE WHEN bond_type = ' = ' THEN 1 ELSE 0 END) * 100.0 / COUNT(bond_id)) AS percent FROM bonds T1 WHERE T1.molecule_id = 'TR047' AND T1.bond_type IS NOT NULL"}


14it [01:59,  8.33s/it]

13 :  {'chain_of_thought_reasoning': "To answer the question, I need to find the user who added a bounty of 50 to a post with a title mentioning 'variance'. First, I need to identify the relevant tables and columns. The 'BountyAmount' column in a table (likely 'Bounties') should be checked for the value 50. The 'Title' column in a 'Posts' table should contain the word 'variance'. These two tables are likely connected via a 'PostId'. After identifying the correct post, I need to find the user who added the bounty, which involves joining with a 'Users' table on a 'UserId' to get the 'DisplayName'. I will use JOINs to connect these tables appropriately, filter with WHERE clauses for the bounty amount and post title, and select only the 'DisplayName' as per the instructions.", 'SQL': "SELECT T3.DisplayName FROM Bounties T1 JOIN Posts T2 ON T1.PostId = T2.PostId JOIN Users T3 ON T1.UserId = T3.UserId WHERE T1.BountyAmount = 50 AND T2.Title LIKE '%variance%'"}


15it [02:06,  8.45s/it]

14 :  {'chain_of_thought_reasoning': "To answer the question, I need to find the dates when users from Rochester, NY obtained their badges. This requires joining two tables: one that contains user information (including location and user ID) and another that contains badge information (including the user ID and the date the badge was obtained). First, I will identify the table and column where user locations are stored, and filter for users in 'Rochester, NY'. Next, I will join this filtered list of users with the table containing badge information, using the user ID as the common key. Finally, I will select the date column from the badge information table, ensuring that I only include dates in the output, as per the database admin instructions.", 'SQL': "SELECT T2.DateObtained FROM Users T1 JOIN Badges T2 ON T1.UserId = T2.UserId WHERE T1.Location = 'Rochester, NY'"}


In [6]:
from pathlib import Path

schema_path = Path('C:/Users/yousf/Bureau/ConvergenceAI/CHESS_Impl/data/test/generated_candidate.json')
with open(schema_path, 'w') as f:
    json.dump(res, f, indent=4)

## Cost Estimation per task

In [5]:

PROMPT_PATH = os.getenv("PROMPT_ROOT_PATH") + "\\candidate_generation.txt"
prompt = load_prompt(PROMPT_PATH)

In [6]:

def tokens_calc(example):
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(example))
    return num_tokens

In [7]:
#prompt template tokens 
tokens_calc(prompt)

631

The prompt template has 631 tokens in total, and we have also 3 other variables (Database_Schema,Question and Hint).
After i see a formatted prompt example it contains about 2000 tokens (because database_schema will be long)
So let's suppose that input tokens is 2000

In [8]:
## Output tokens estimation from an example 
output_example = """
{'chain_of_thought_reasoning': "First, I identified the relevant columns from the question: CDSCode, County Code, School Type, Low Grade, High Grade, and County Name. Then, I determined that the question is asking for the city location of a high school level school with Lunch Provision 2, whose lowest grade is 9 and the highest grade is 12 in the county of Merced. I used the hint that high school can be represented as EILCode = 'HS'. I joined the frpm and schools tables based on the CDSCode, and then filtered the results to match the conditions specified in the question. Finally, I selected the City column from the schools table, which is the column that provides the city location.",
 'SQL': "SELECT City FROM schools WHERE EILCode = 'HS' AND County = 'Merced' AND LowGrade = '9' AND HighGrade = '12' AND NSLPProvisionStatus = 'Lunch Provision 2' AND CDSCode IN (SELECT CDSCode FROM frpm WHERE CountyCode = '02');"}
"""
tokens_calc(output_example)

227

Let's suppose the output tokens is 250

In [9]:
## Price calculation (just with gpt4 because in this module we don't use gpt3.5) 
input_price_per_token_gpt4 = 0.01 / 1000
output_price_per_token_gpt4 = 0.03 / 1000
price_gpt4 = 2000 * input_price_per_token_gpt4 + 250 * output_price_per_token_gpt4
print("estimated price per Task (GPT-4):", price_gpt4, "$")

estimated price per Task (GPT-4): 0.0275 $


In [10]:
## in this module there is a number of retrials so let's estimate the price with different number of retrials 

num_retrials = [1, 2, 3, 4, 5]
for num_retrial in num_retrials:
    total_price = price_gpt4 * num_retrial
    print("The estimated price with number of retrials of " + str(num_retrial) + ": ", total_price)

The estimated price with number of retrials of 1:  0.0275
The estimated price with number of retrials of 2:  0.055
The estimated price with number of retrials of 3:  0.0825
The estimated price with number of retrials of 4:  0.11
The estimated price with number of retrials of 5:  0.1375
